#### pyproject.toml

In [236]:
%%writefile pyproject.toml
[build-system]
requires = ["setuptools>=68", "wheel"]
build-backend = "setuptools.build_meta"

[project]
name = "tik"
version = "0.1.0"
description = "The Investigation Kit (Phase 1) — Orwell-style 2-pane PyQt6 desktop app"
readme = "README.md"
requires-python = ">=3.10"
authors = [{ name = "DevPilot" }]
dependencies = [
    "PyQt6>=6.6",
    "PyQt6-WebEngine>=6.6",
    "pydantic>=2.6,<3",
    "loguru>=0.7",
    "httpx>=0.27",
]

[project.optional-dependencies]
dev = ["pytest>=8", "pytest-qt>=4", "ruff>=0.5"]

[tool.setuptools]
packages = ["tik"]

[tool.setuptools.package-data]
tik = [
    "ui/web/*.html",
    "ui/web/*.css",
    "ui/web/*.js",
    "data/*.json",
    "data/docs/*.html",
    "data/chunks/*.json",
]

[tool.pytest.ini_options]
testpaths = ["tests"]
python_files = ["test_*.py"]


Overwriting pyproject.toml


#### README.md

In [237]:
%%writefile README.md
# The Investigation Kit (TIK) — Phase 1

Orwell-style 2-pane desktop app built with **PyQt6**. Phase 1 focuses on local JSON seeds and a thin service layer you can later swap for an HTTP API.

## Features (Phase 1)
- Left: **Profiler** (Subject card + field drop zones) and **Relationship Graph (stub)** via a tab.
- Right: **Reader** (sources, documents, HTML viewer with highlighted datachunks). Listener/Insider are stubbed.
- Drag a highlighted **datachunk** into a matching Profiler field to accept it.
- **Conflict resolver** when an incoming chunk collides by `exclusiveGroup` on the same field.
- **Objectives** modal evaluates simple predicates (AND/OR of “field exists”).
- **Advisor/Log** dock shows periodic events (timer-mock now; WebSocket/API later).
- **Undo/Redo** via `QUndoStack` for accept and resolve.

## Quickstart

```bash
python -m venv .venv
# Windows
.venv\Scripts\activate
# macOS/Linux
source .venv/bin/activate

pip install -U pip
pip install -e .
# or: pip install -r requirements.txt

#run demo
python -m tik.app
# or
python scripts/run_demo.py

# run tests
pytest -q

#structure
tik/
  core/…        # models, store, commands, services
  ui/…          # Qt widgets, overlays, web assets
  data/…        # seed case, docs, chunks
scripts/
tests/

Overwriting README.md


#### requirements.txt

In [238]:
%%writefile requirements.txt
PyQt6>=6.6
PyQt6-WebEngine>=6.6
pydantic>=2.6,<3
loguru>=0.7
httpx>=0.27
pytest>=8
pytest-qt>=4

Overwriting requirements.txt


In [239]:
%%writefile tik/__init__.py
from __future__ import annotations

__all__ = ["__version__"]
__version__ = "0.1.0"


Overwriting tik/__init__.py


## Tik

#### tik/app.py

In [240]:
%%writefile tik/app.py
from __future__ import annotations

import sys
from pathlib import Path
from loguru import logger

from PyQt6.QtCore import QTimer, Qt
from PyQt6.QtWidgets import QApplication

from .theme.qss import apply_theme
from .core.store import Store
from .core.services.fake import FakeCaseService, FakeDocumentService, FakeChunkService, FakeObjectiveService, FakeEventService
from .ui.shell import MainWindow


def main() -> int:
    app = QApplication(sys.argv)
    app.setApplicationName("The Investigation Kit")
    app.setOrganizationName("TIK")
    apply_theme(app)

    # Services (can be swapped by API services later)
    base_dir = Path(__file__).resolve().parent
    data_dir = base_dir / "data"
    case_svc = FakeCaseService(data_dir)
    doc_svc = FakeDocumentService(data_dir)
    chunk_svc = FakeChunkService(data_dir)
    obj_svc = FakeObjectiveService()
    evt_svc = FakeEventService()

    store = Store(case_service=case_svc, document_service=doc_svc, chunk_service=chunk_svc,
                  objective_service=obj_svc, event_service=evt_svc)

    # Bootstrap default case
    store.load_default_case()

    # UI
    win = MainWindow(store)
    win.resize(1280, 800)
    win.show()

    # Advisor event poller (timer-mock)
    timer = QTimer(win)
    timer.setInterval(3000)  # 3s
    timer.timeout.connect(store.poll_events)
    timer.start()

    logger.info("TIK started")
    return app.exec()


if __name__ == "__main__":
    raise SystemExit(main())


Overwriting tik/app.py


#### tik/theme/qss.py

In [241]:
%%writefile tik/theme/qss.py
from __future__ import annotations

from PyQt6.QtWidgets import QApplication


TOKENS = {
    "bg": "#0e141b",
    "panel": "#111922",
    "surface": "#16202b",
    "text": "#d7e3f4",
    "muted": "#93a4b8",
    "accent": "#58a6ff",
    "accent2": "#f778ba",
    "danger": "#ff6b6b",
    "ok": "#4cc38a",
    "warn": "#ffcc66",
    "chip": "#1f2a36",
    "chip_border": "#2a3a4a",
}


def stylesheet() -> str:
    t = TOKENS
    return f"""
    * {{
        color: {t['text']};
        font-family: Segoe UI, Inter, Arial;
    }}
    QMainWindow, QWidget {{
        background: {t['bg']};
    }}
    QSplitter::handle {{
        background: {t['panel']};
        width: 6px;
    }}
    QTabBar::tab {{
        background: {t['panel']};
        padding: 6px 10px;
        margin: 2px;
        border-radius: 6px;
    }}
    QTabBar::tab:selected {{
        background: {t['surface']};
        color: {t['accent']};
    }}
    QListView, QTreeView, QTextEdit {{
        background: {t['panel']};
        border: 1px solid {t['chip_border']};
        border-radius: 8px;
    }}
    QToolBar {{
        background: {t['surface']};
        border: none;
    }}
    QPushButton {{
        background: {t['chip']};
        border: 1px solid {t['chip_border']};
        border-radius: 8px;
        padding: 6px 12px;
    }}
    QPushButton:hover {{ border-color: {t['accent']}; }}
    QLabel#Chip {{
        background: {t['chip']};
        border: 1px solid {t['chip_border']};
        border-radius: 8px;
        padding: 2px 6px;
        color: {t['muted']};
    }}
    """


def apply_theme(app: QApplication) -> None:
    app.setStyleSheet(stylesheet())


Overwriting tik/theme/qss.py


#### tik/core/models.py

In [242]:
%%writefile tik/core/models.py
from __future__ import annotations

from typing import List, Optional, Dict, Literal
from pydantic import BaseModel, Field


class Person(BaseModel):
    id: str
    name: Optional[str] = None
    dob: Optional[str] = None
    address: Optional[str] = None
    occupation: Optional[str] = None
    # accepted values indexed by field
    accepted: Dict[str, "AcceptedChunk"] = Field(default_factory=dict)


class Source(BaseModel):
    id: str
    title: str


class Document(BaseModel):
    id: str
    source_id: str
    title: str
    path: str  # relative package path to HTML file


class DataChunk(BaseModel):
    id: str
    document_id: str
    source_id: str
    field: str                 # e.g., "name", "dob"
    value: str
    offset_start: int
    offset_end: int
    exclusive_group: Optional[str] = None  # conflicts within same field


class AcceptedChunk(BaseModel):
    chunk_id: str
    field: str
    value: str
    source_id: str
    document_id: str
    exclusive_group: Optional[str] = None


class Case(BaseModel):
    id: str
    title: str
    people: List[Person]
    sources: List[Source]
    documents: List[Document]


class ObjectivePredicate(BaseModel):
    """Minimal predicate system: op in {'exists'} with a path like 'person.name'."""
    op: Literal["exists"]
    path: str  # "person.<field>"


class ObjectiveExpr(BaseModel):
    """Boolean tree: kind 'AND' or 'OR' with either children or a predicate leaf."""
    kind: Literal["AND", "OR", "LEAF"]
    children: Optional[List["ObjectiveExpr"]] = None
    predicate: Optional[ObjectivePredicate] = None


class Objective(BaseModel):
    id: str
    title: str
    expr: ObjectiveExpr
    satisfied: bool = False


class AdvisorEvent(BaseModel):
    id: str
    text: str
    level: Literal["info", "warn", "error"] = "info"


Overwriting tik/core/models.py


#### tik/core/store.py

In [243]:
%%writefile tik/core/store.py
from __future__ import annotations

from dataclasses import dataclass
from typing import Callable, Dict, Optional, Tuple

from loguru import logger
from PyQt6.QtCore import QObject, pyqtSignal
from PyQt6.QtGui import QUndoStack

from .models import Case, Person, Source, Document, DataChunk, AcceptedChunk, Objective, AdvisorEvent
from .commands import AcceptChunkCommand, ResolveConflictCommand, RetractChunkCommand
from .services.base import CaseService, DocumentService, ChunkService, ObjectiveService, EventService


@dataclass
class Selection:
    case: Optional[Case] = None
    person: Optional[Person] = None
    source: Optional[Source] = None
    document: Optional[Document] = None


class Store(QObject):
    # Signals for UI
    selectionChanged = pyqtSignal()
    documentLoaded = pyqtSignal(str, list)  # (html, chunks_json_str)
    acceptedChanged = pyqtSignal()
    objectivesChanged = pyqtSignal()
    advisorEvent = pyqtSignal(object)  # AdvisorEvent

    def __init__(self, *, case_service: CaseService, document_service: DocumentService,
                 chunk_service: ChunkService, objective_service: ObjectiveService,
                 event_service: EventService):
        super().__init__()
        self.case_svc = case_service
        self.doc_svc = document_service
        self.chunk_svc = chunk_service
        self.obj_svc = objective_service
        self.evt_svc = event_service

        self.sel = Selection()
        self.undo_stack = QUndoStack(self)
        self._conflict_resolver: Optional[Callable[[AcceptedChunk, DataChunk], Optional[AcceptedChunk]]] = None

    # === Bootstrapping & selection ===

    def load_default_case(self) -> None:
        case = self.case_svc.load_default_case()
        self.sel.case = case
        self.sel.person = case.people[0] if case.people else None
        self.selectionChanged.emit()
        logger.info("Loaded case {}", case.title)

    def set_conflict_resolver(self, fn: Callable[[AcceptedChunk, DataChunk], Optional[AcceptedChunk]]) -> None:
        self._conflict_resolver = fn

    def select_source(self, source: Optional[Source]) -> None:
        self.sel.source = source
        self.selectionChanged.emit()

    def select_document(self, doc: Optional[Document]) -> None:
        self.sel.document = doc
        if not doc:
            return
        html, chunks = self.doc_svc.load_document_html_and_chunks(doc.id)
        self.documentLoaded.emit(html, chunks)
        self.selectionChanged.emit()

    # === Accept / retract / conflicts ===

    def request_accept(self, chunk: DataChunk) -> None:
        person = self.sel.person
        if not person:
            return
        current = person.accepted.get(chunk.field)
        if current and current.exclusive_group and chunk.exclusive_group and current.exclusive_group == chunk.exclusive_group:
            # No-op if identical chunk
            if current.chunk_id == chunk.id:
                logger.debug("Accept idempotent for chunk {}", chunk.id)
                return
            logger.debug("Conflict detected on field {} between {} and {}", chunk.field, current.chunk_id, chunk.id)
            winner: Optional[AcceptedChunk] = None
            if self._conflict_resolver:
                winner = self._conflict_resolver(current, chunk)
            if winner is None:
                logger.debug("Conflict unresolved; abort")
                return
            cmd = ResolveConflictCommand(person, current, chunk, winner)
            self.undo_stack.push(cmd)
        else:
            cmd = AcceptChunkCommand(person, chunk)
            self.undo_stack.push(cmd)

        # Objectives re-eval
        self.evaluate_objectives()
        self.acceptedChanged.emit()

    def retract_field(self, field: str) -> None:
        person = self.sel.person
        if not person or field not in person.accepted:
            return
        cmd = RetractChunkCommand(person, field)
        self.undo_stack.push(cmd)
        self.evaluate_objectives()
        self.acceptedChanged.emit()

    # === Objectives & events ===

    def evaluate_objectives(self) -> None:
        case = self.sel.case
        person = self.sel.person
        if not case or not person:
            return
        changed = self.obj_svc.evaluate(case, person)
        if changed:
            self.objectivesChanged.emit()

    def poll_events(self) -> None:
        evt = self.evt_svc.poll()
        if evt:
            self.advisorEvent.emit(evt)
            logger.info("Advisor: {}", evt.text)


Overwriting tik/core/store.py


#### tik/core/commands.py

In [244]:
%%writefile tik/core/commands.py
from __future__ import annotations

from PyQt6.QtGui import QUndoCommand

from .models import Person, DataChunk, AcceptedChunk


class AcceptChunkCommand(QUndoCommand):
    def __init__(self, person: Person, chunk: DataChunk):
        super().__init__(f"Accept {chunk.field}")
        self.person = person
        self.chunk = chunk
        self._prev: AcceptedChunk | None = person.accepted.get(chunk.field)

    def redo(self) -> None:
        self.person.accepted[self.chunk.field] = AcceptedChunk(
            chunk_id=self.chunk.id,
            field=self.chunk.field,
            value=self.chunk.value,
            source_id=self.chunk.source_id,
            document_id=self.chunk.document_id,
            exclusive_group=self.chunk.exclusive_group,
        )

    def undo(self) -> None:
        if self._prev is None:
            self.person.accepted.pop(self.chunk.field, None)
        else:
            self.person.accepted[self._prev.field] = self._prev


class ResolveConflictCommand(QUndoCommand):
    """Winner is either the existing accepted or the incoming chunk turned into an AcceptedChunk."""
    def __init__(self, person: Person, current: AcceptedChunk, incoming: DataChunk, winner: AcceptedChunk):
        super().__init__(f"Resolve conflict {incoming.field}")
        self.person = person
        self.current = current
        self.incoming = incoming
        self.winner = winner
        self._before = current

    def redo(self) -> None:
        self.person.accepted[self.winner.field] = self.winner

    def undo(self) -> None:
        self.person.accepted[self._before.field] = self._before


class RetractChunkCommand(QUndoCommand):
    def __init__(self, person: Person, field: str):
        super().__init__(f"Retract {field}")
        self.person = person
        self.field = field
        self._prev = person.accepted.get(field)

    def redo(self) -> None:
        self.person.accepted.pop(self.field, None)

    def undo(self) -> None:
        if self._prev:
            self.person.accepted[self.field] = self._prev


Overwriting tik/core/commands.py


#### tik/core/services/base.py

In [245]:
%%writefile tik/core/services/base.py
from __future__ import annotations

from abc import ABC, abstractmethod
from typing import List, Tuple, Optional

from ..models import Case, Document, DataChunk, Objective, AdvisorEvent, Person


class CaseService(ABC):
    @abstractmethod
    def load_default_case(self) -> Case: ...


class DocumentService(ABC):
    @abstractmethod
    def load_document_html_and_chunks(self, document_id: str) -> Tuple[str, list]: ...


class ChunkService(ABC):
    @abstractmethod
    def list_chunks_for_document(self, document_id: str) -> List[DataChunk]: ...


class ObjectiveService(ABC):
    @abstractmethod
    def evaluate(self, case: Case, person: Person) -> bool:
        """Recompute objective statuses. Return True if any change."""


class EventService(ABC):
    @abstractmethod
    def poll(self) -> Optional[AdvisorEvent]: ...


Overwriting tik/core/services/base.py


#### tik/core/services/fake.py

In [246]:
%%writefile tik/core/services/fake.py
from __future__ import annotations

import json
import itertools
from pathlib import Path
from typing import List, Tuple, Optional

from pydantic import TypeAdapter

from ..models import (
    Case, Person, Source, Document, DataChunk, Objective, ObjectiveExpr, ObjectivePredicate, AdvisorEvent, AcceptedChunk
)
from ..document_renderer import wrap_chunks_into_html
from .base import CaseService, DocumentService, ChunkService, ObjectiveService, EventService


class FakeCaseService(CaseService):
    def __init__(self, data_dir: Path):
        self.data_dir = data_dir
        self._case: Optional[Case] = None
        self._objectives: List[Objective] = []

    def load_default_case(self) -> Case:
        seed_path = self.data_dir / "seed_case.json"
        data = json.loads(seed_path.read_text(encoding="utf-8"))
        case = Case.model_validate(data["case"])
        # Load objectives
        self._objectives = TypeAdapter(List[Objective]).validate_python(data.get("objectives", []))
        # Attach objectives back to case via private attr for retrieval by ObjectiveService
        case._objectives = self._objectives  # type: ignore[attr-defined]
        self._case = case
        return case


class FakeDocumentService(DocumentService):
    def __init__(self, data_dir: Path):
        self.data_dir = data_dir

    def load_document_html_and_chunks(self, document_id: str) -> Tuple[str, list]:
        doc_html_path = self.data_dir / "docs" / f"{document_id}.html"
        chunks_path = self.data_dir / "chunks" / f"{document_id}.json"
        html = doc_html_path.read_text(encoding="utf-8")
        chunks = TypeAdapter(List[DataChunk]).validate_python(json.loads(chunks_path.read_text(encoding="utf-8")))
        wrapped_html = wrap_chunks_into_html(html, chunks)
        # For WebEngine, also pass chunks as JSON list for client-side DnD attrs (redundant but handy to inspect)
        return wrapped_html, [c.model_dump() for c in chunks]


class FakeChunkService(ChunkService):
    def __init__(self, data_dir: Path):
        self.data_dir = data_dir

    def list_chunks_for_document(self, document_id: str) -> List[DataChunk]:
        p = self.data_dir / "chunks" / f"{document_id}.json"
        return TypeAdapter(List[DataChunk]).validate_python(json.loads(p.read_text(encoding="utf-8")))


class FakeObjectiveService(ObjectiveService):
    def evaluate(self, case: Case, person: Person) -> bool:
        changed = False
        objectives: List[Objective] = getattr(case, "_objectives", [])
        for obj in objectives:
            was = obj.satisfied
            obj.satisfied = self._eval_expr(obj.expr, person)
            changed = changed or (was != obj.satisfied)
        return changed

    def _eval_expr(self, expr: ObjectiveExpr, person: Person) -> bool:
        if expr.kind == "LEAF" and expr.predicate:
            pred = expr.predicate
            if pred.op == "exists":
                _, field = pred.path.split(".", 1)
                return field in person.accepted and bool(person.accepted[field].value)
            return False
        if expr.kind == "AND":
            return all(self._eval_expr(c, person) for c in expr.children or [])
        if expr.kind == "OR":
            return any(self._eval_expr(c, person) for c in expr.children or [])
        return False


class FakeEventService(EventService):
    _counter = itertools.count(1)

    def poll(self) -> Optional[AdvisorEvent]:
        i = next(self._counter)
        levels = ["info", "warn", "error"]
        return AdvisorEvent(id=str(i), text=f"Background check #{i} completed.", level=levels[i % 3])


Overwriting tik/core/services/fake.py


#### tik/core/services/api.py

In [247]:
%%writefile tik/core/services/api.py
from __future__ import annotations

import httpx
from typing import List, Tuple, Optional

from .base import CaseService, DocumentService, ChunkService, ObjectiveService, EventService
from ..models import Case, Document, DataChunk, AdvisorEvent, Person


class ApiCaseService(CaseService):
    def __init__(self, base_url: str, client: Optional[httpx.Client] = None):
        self.base_url = base_url.rstrip("/")
        self.client = client or httpx.Client(timeout=10.0)

    def load_default_case(self) -> Case:
        raise NotImplementedError("HTTP API not implemented in Phase 1")


class ApiDocumentService(DocumentService):
    def __init__(self, base_url: str, client: Optional[httpx.Client] = None):
        self.base_url = base_url.rstrip("/")
        self.client = client or httpx.Client(timeout=10.0)

    def load_document_html_and_chunks(self, document_id: str) -> Tuple[str, list]:
        raise NotImplementedError("HTTP API not implemented in Phase 1")


class ApiChunkService(ChunkService):
    def __init__(self, base_url: str, client: Optional[httpx.Client] = None):
        self.base_url = base_url.rstrip("/")
        self.client = client or httpx.Client(timeout=10.0)

    def list_chunks_for_document(self, document_id: str) -> List[DataChunk]:
        raise NotImplementedError("HTTP API not implemented in Phase 1")


class ApiObjectiveService(ObjectiveService):
    def __init__(self, base_url: str, client: Optional[httpx.Client] = None):
        self.base_url = base_url.rstrip("/")
        self.client = client or httpx.Client(timeout=10.0)

    def evaluate(self, case: Case, person: Person) -> bool:
        raise NotImplementedError("HTTP API not implemented in Phase 1")


class ApiEventService(EventService):
    def __init__(self, base_url: str, client: Optional[httpx.Client] = None):
        self.base_url = base_url.rstrip("/")
        self.client = client or httpx.Client(timeout=10.0)

    def poll(self) -> Optional[AdvisorEvent]:
        raise NotImplementedError("HTTP API not implemented in Phase 1")


Overwriting tik/core/services/api.py


#### tik/core/document_renderer.py

In [272]:
%%writefile tik/core/document_renderer.py
from __future__ import annotations

from html import escape
from typing import List, Tuple

from .models import DataChunk


def _safe_html(html: str) -> str:
    """
    Minimal sanitizer: escape all tags except a tiny allowlist we rely on for basic formatting.
    NOTE: Backend should still sanitize before persisting.
    """
    allowed = {"b", "i", "em", "strong", "u", "p", "br", "ul", "ol", "li", "span"}
    out: list[str] = []
    i = 0
    while i < len(html):
        ch = html[i]
        if ch == "<":
            j = html.find(">", i + 1)
            if j == -1:
                out.append(escape(html[i:]))
                break
            tag = html[i + 1 : j].strip().strip("/")
            tag_name = tag.split()[0].lower() if tag else ""
            if tag_name in allowed:
                out.append(html[i : j + 1])
            else:
                out.append(escape(html[i : j + 1]))
            i = j + 1
        else:
            out.append(escape(ch))
            i += 1
    return "".join(out)


def _compute_ranges_with_fallback(safe: str, chunks: List[DataChunk]) -> List[Tuple[int, int, DataChunk]]:
    """
    Build non-overlapping (start, end, chunk) ranges on the sanitized `safe` string.

    Strategy:
      1) Trust offsets if they are within bounds AND the substring contains chunk.value.
      2) Otherwise, fallback to searching `chunk.value` inside `safe` (first non-overlapping match).
         This fixes common offset drift caused by HTML tags vs textContent indexing.
    """
    ranges: List[Tuple[int, int, DataChunk]] = []

    def overlaps(a_start: int, a_end: int) -> bool:
        for s, e, _ in ranges:
            if max(a_start, s) < min(a_end, e):
                return True
        return False

    for c in chunks:
        start, end = c.offset_start, c.offset_end
        ok = isinstance(start, int) and isinstance(end, int) and 0 <= start < end <= len(safe)
        if ok:
            segment = safe[start:end]
            if c.value not in segment:
                ok = False

        if not ok:
            # Fallback: locate `value` directly in `safe`, avoiding already-reserved ranges
            needle = c.value or ""
            if not needle:
                continue
            search_pos = 0
            found = False
            while True:
                idx = safe.find(needle, search_pos)
                if idx == -1:
                    break
                candidate = (idx, idx + len(needle))
                if not overlaps(*candidate):
                    start, end = candidate
                    found = True
                    break
                search_pos = idx + 1
            if not found:
                # Give up on this chunk if we cannot locate it
                continue

        if not overlaps(start, end):
            ranges.append((start, end, c))

    ranges.sort(key=lambda t: t[0])
    return ranges


def wrap_chunks_into_html(html: str, chunks: List[DataChunk]) -> str:
    """
    Return an HTML fragment with <span class="chunk" draggable="true" ...> inserted
    exactly around chunk values.

    Steps:
      1) Sanitize (keep a few formatting tags).
      2) Compute non-overlapping highlight ranges using offsets OR fallback string search.
      3) Insert chunk spans in order to keep HTML valid (no broken nesting).
    """
    safe = _safe_html(html)
    ranges = _compute_ranges_with_fallback(safe, chunks)

    result: list[str] = []
    cursor = 0
    for start, end, c in ranges:
        if cursor < start:
            result.append(safe[cursor:start])
        data_attrs = (
            f'data-chunk-id="{c.id}" '
            f'data-field="{c.field}" '
            f'data-value="{escape(c.value)}" '
            f'data-source-id="{c.source_id}" '
            f'data-document-id="{c.document_id}" '
            f'data-exclusive-group="{escape(c.exclusive_group or "")}"'
        )
        result.append(f'<span class="chunk" draggable="true" {data_attrs}>')
        result.append(safe[start:end])
        result.append("</span>")
        cursor = end

    if cursor < len(safe):
        result.append(safe[cursor:])

    return "".join(result)


Overwriting tik/core/document_renderer.py


#### tik/ui/shell.py

In [249]:
%%writefile tik/ui/shell.py
from __future__ import annotations

from pathlib import Path

from PyQt6.QtCore import Qt, QUrl
from PyQt6.QtWidgets import (
    QMainWindow, QWidget, QSplitter, QTabWidget, QToolBar, QFileDialog, QMessageBox
)
from PyQt6.QtGui import QAction
from ..core.store import Store
from .left.profiler import ProfilerPanel
from .left.graph_stub import GraphPanel
from .right.reader import ReaderPanel
from .overlays.objectives import ObjectivesDialog
from .overlays.conflicts import ConflictsDialog
from .overlays.logdock import LogDock
from .widgets.toast import Toast


class MainWindow(QMainWindow):
    def __init__(self, store: Store):
        super().__init__()
        self.setWindowTitle("The Investigation Kit")
        self.store = store
        self.toast = Toast(self)

        left_tabs = QTabWidget()
        self.profiler = ProfilerPanel(store)
        self.graph = GraphPanel()
        left_tabs.addTab(self.profiler, "Profiler")
        left_tabs.addTab(self.graph, "Graph")

        self.reader = ReaderPanel(store)

        split = QSplitter()
        split.setOrientation(Qt.Orientation.Horizontal)
        split.addWidget(left_tabs)
        split.addWidget(self.reader)
        split.setStretchFactor(0, 0)
        split.setStretchFactor(1, 1)
        self.setCentralWidget(split)

        self.logdock = LogDock(self)
        self.addDockWidget(Qt.DockWidgetArea.BottomDockWidgetArea, self.logdock)

        self._build_toolbar()
        self._wire_signals()

        # Provide conflict resolver to store (UI mediation)
        def resolver(current, incoming):
            dlg = ConflictsDialog(self, current=current, incoming=incoming)
            if dlg.exec():
                return dlg.winner()
            return None

        self.store.set_conflict_resolver(resolver)

    def _build_toolbar(self) -> None:
        tb = QToolBar("Main")
        self.addToolBar(tb)

        act_obj = QAction("Objectives", self)
        act_obj.triggered.connect(self._open_objectives)
        tb.addAction(act_obj)

        act_undo = self.store.undo_stack.createUndoAction(self, "Undo")
        act_redo = self.store.undo_stack.createRedoAction(self, "Redo")
        tb.addAction(act_undo)
        tb.addAction(act_redo)

    def _wire_signals(self) -> None:
        self.store.advisorEvent.connect(lambda evt: (self.toast.show(evt.text), self.logdock.append(evt)))
        self.store.objectivesChanged.connect(lambda: self.logdock.append_text("Objectives updated"))

    def _open_objectives(self) -> None:
        dlg = ObjectivesDialog(self, self.store)
        dlg.exec()


Overwriting tik/ui/shell.py


#### tik/ui/left/profiler.py

In [250]:
%%writefile tik/ui/left/profiler.py
from __future__ import annotations

from typing import List

from PyQt6.QtCore import Qt
from PyQt6.QtWidgets import QWidget, QVBoxLayout, QLabel, QGridLayout, QPushButton, QHBoxLayout

from ...core.store import Store
from ...core.models import AcceptedChunk
from ..widgets.drop_zone import FieldDropZone


FIELDS: List[str] = ["name", "dob", "address", "occupation"]


class ProfilerPanel(QWidget):
    def __init__(self, store: Store):
        super().__init__()
        self.store = store
        self._labels: dict[str, QLabel] = {}
        self._zones: dict[str, FieldDropZone] = {}

        lay = QVBoxLayout(self)
        title = QLabel("Subject")
        title.setStyleSheet("font-size: 20px; font-weight: 600;")
        lay.addWidget(title)

        grid = QGridLayout()
        row = 0
        for field in FIELDS:
            lab = QLabel(field.capitalize())
            lab.setObjectName("Chip")
            val = QLabel("—")
            val.setWordWrap(True)

            dz = FieldDropZone(field)
            dz.acceptRequested.connect(self._on_accept)

            self._labels[field] = val
            self._zones[field] = dz

            grid.addWidget(lab, row, 0)
            grid.addWidget(val, row, 1)
            grid.addWidget(dz, row, 2)
            row += 1

        lay.addLayout(grid)

        btns = QHBoxLayout()
        for field in FIELDS:
            b = QPushButton(f"Retract {field}")
            b.clicked.connect(lambda _, f=field: self.store.retract_field(f))
            btns.addWidget(b)
        lay.addLayout(btns)

        self.store.acceptedChanged.connect(self._refresh)
        self.store.selectionChanged.connect(self._refresh)
        self._refresh()

    def _on_accept(self, chunk_payload) -> None:
        self.store.request_accept(chunk_payload)

    def _refresh(self) -> None:
        person = self.store.sel.person
        if not person:
            return
        for f in FIELDS:
            ac: AcceptedChunk | None = person.accepted.get(f)
            self._labels[f].setText(ac.value if ac else "—")


Overwriting tik/ui/left/profiler.py


#### tik/ui/left/graph_stub.py

In [251]:
%%writefile tik/ui/left/graph_stub.py
from __future__ import annotations

from PyQt6.QtWidgets import QWidget, QVBoxLayout, QLabel


class GraphPanel(QWidget):
    def __init__(self):
        super().__init__()
        lay = QVBoxLayout(self)
        lab = QLabel("Relationship Graph (stub)")
        lab.setStyleSheet("font-size: 16px; color: #93a4b8;")
        lay.addWidget(lab)

    # API placeholders for future graph integration
    def set_graph_data(self, data) -> None:
        pass

    def center_on(self, node_id: str) -> None:
        pass


Overwriting tik/ui/left/graph_stub.py


#### tik/ui/right/reader.py

In [252]:
%%writefile tik/ui/right/reader.py
from __future__ import annotations

import json
from pathlib import Path
from importlib import resources

from PyQt6.QtCore import Qt, QUrl
from PyQt6.QtWidgets import QWidget, QVBoxLayout, QHBoxLayout, QLabel, QListView, QSplitter
from PyQt6.QtWebEngineWidgets import QWebEngineView

from ...core.store import Store
from .widgets.source_list import SourceListModel, SourceListView
from .widgets.document_list import DocumentListModel, DocumentListView


class DocumentView(QWebEngineView):
    def __init__(self):
        super().__init__()
        self.setContextMenuPolicy(Qt.ContextMenuPolicy.NoContextMenu)

    def set_document(self, html_fragment: str) -> None:
        # Build a small HTML page that links our local CSS/JS assets.
        base = resources.files("tik.ui.web")
        css = QUrl.fromLocalFile(str(base.joinpath("style.css")))
        js = QUrl.fromLocalFile(str(base.joinpath("highlight.js")))
        tpl = f"""<!doctype html>
<html><head>
<meta charset="utf-8">
<link rel="stylesheet" href="{css.toString()}">
</head>
<body>
<div id="container">{html_fragment}</div>
<script src="{js.toString()}"></script>
</body></html>"""
        self.setHtml(tpl)


class ReaderPanel(QWidget):
    def __init__(self, store: Store):
        super().__init__()
        self.store = store

        lay = QHBoxLayout(self)
        splitter = QSplitter(self)
        splitter.setOrientation(Qt.Orientation.Horizontal)

        self.source_model = SourceListModel(store)
        self.source_view = SourceListView()
        self.source_view.setModel(self.source_model)

        self.doc_model = DocumentListModel(store)
        self.doc_view = DocumentListView()
        self.doc_view.setModel(self.doc_model)

        left = QWidget()
        l = QVBoxLayout(left)
        l.addWidget(QLabel("Sources"))
        l.addWidget(self.source_view)
        l.addWidget(QLabel("Documents"))
        l.addWidget(self.doc_view)

        self.web = DocumentView()

        splitter.addWidget(left)
        splitter.addWidget(self.web)
        splitter.setStretchFactor(0, 0)
        splitter.setStretchFactor(1, 1)

        lay.addWidget(splitter)

        self._wire()

    def _wire(self) -> None:
        self.source_view.sourceSelected.connect(self.store.select_source)
        self.doc_view.documentSelected.connect(self.store.select_document)
        self.store.documentLoaded.connect(self._on_document_loaded)

    def _on_document_loaded(self, html, _chunks_json_list) -> None:
        self.web.set_document(html)


Overwriting tik/ui/right/reader.py


#### tik/ui/right/widgets/source_list.py

In [253]:
%%writefile tik/ui/right/widgets/source_list.py
from __future__ import annotations

from typing import List, Optional

from PyQt6.QtCore import Qt, QAbstractListModel, QModelIndex, pyqtSignal
from PyQt6.QtWidgets import QListView
from PyQt6.QtCore import QItemSelectionModel

from ....core.models import Source
from ....core.store import Store


class SourceListModel(QAbstractListModel):
    def __init__(self, store: Store):
        super().__init__()
        self.store = store
        self._items: List[Source] = store.sel.case.sources if store.sel.case else []
        store.selectionChanged.connect(self._reload)

    def _reload(self) -> None:
        self.beginResetModel()
        self._items = self.store.sel.case.sources if self.store.sel.case else []
        self.endResetModel()

    def rowCount(self, parent=QModelIndex()) -> int:
        return 0 if parent.isValid() else len(self._items)

    def data(self, index: QModelIndex, role=Qt.ItemDataRole.DisplayRole):
        if not index.isValid():
            return None
        src = self._items[index.row()]
        if role == Qt.ItemDataRole.DisplayRole:
            return src.title
        return None

    def source(self, row: int) -> Source:
        return self._items[row]


class SourceListView(QListView):
    sourceSelected = pyqtSignal(object)  # Source | None

    def __init__(self):
        super().__init__()
        self.setSelectionMode(QListView.SelectionMode.SingleSelection)
        self._sel_model: Optional[QItemSelectionModel] = None

    def setModel(self, model) -> None:  # type: ignore[override]
        # disconnect old selection model if any
        if self._sel_model is not None:
            try:
                self._sel_model.selectionChanged.disconnect(self._on_sel)
            except TypeError:
                pass
        super().setModel(model)
        # connect new selection model
        self._sel_model = self.selectionModel()
        if self._sel_model is not None:
            self._sel_model.selectionChanged.connect(self._on_sel)

    def _on_sel(self, *_):
        idxs = self.selectedIndexes()
        if not idxs:
            self.sourceSelected.emit(None)
            return
        model: SourceListModel = self.model()  # type: ignore[assignment]
        self.sourceSelected.emit(model.source(idxs[0].row()))


Overwriting tik/ui/right/widgets/source_list.py


#### tik/ui/right/widgets/document_list.py

In [254]:
%%writefile tik/ui/right/widgets/document_list.py
from __future__ import annotations

from typing import List, Optional

from PyQt6.QtCore import Qt, QAbstractListModel, QModelIndex, pyqtSignal
from PyQt6.QtWidgets import QListView
from PyQt6.QtCore import QItemSelectionModel

from ....core.models import Document
from ....core.store import Store


class DocumentListModel(QAbstractListModel):
    def __init__(self, store: Store):
        super().__init__()
        self.store = store
        self._items: List[Document] = store.sel.case.documents if store.sel.case else []
        store.selectionChanged.connect(self._reload)

    def _reload(self) -> None:
        self.beginResetModel()
        self._items = self.store.sel.case.documents if self.store.sel.case else []
        self.endResetModel()

    def rowCount(self, parent=QModelIndex()) -> int:
        return 0 if parent.isValid() else len(self._items)

    def data(self, index: QModelIndex, role=Qt.ItemDataRole.DisplayRole):
        if not index.isValid():
            return None
        doc = self._items[index.row()]
        if role == Qt.ItemDataRole.DisplayRole:
            return doc.title
        return None

    def document(self, row: int) -> Document:
        return self._items[row]


class DocumentListView(QListView):
    documentSelected = pyqtSignal(object)  # Document | None

    def __init__(self):
        super().__init__()
        self.setSelectionMode(QListView.SelectionMode.SingleSelection)
        self._sel_model: Optional[QItemSelectionModel] = None

    def setModel(self, model) -> None:  # type: ignore[override]
        if self._sel_model is not None:
            try:
                self._sel_model.selectionChanged.disconnect(self._on_sel)
            except TypeError:
                pass
        super().setModel(model)
        self._sel_model = self.selectionModel()
        if self._sel_model is not None:
            self._sel_model.selectionChanged.connect(self._on_sel)

    def _on_sel(self, *_):
        idxs = self.selectedIndexes()
        if not idxs:
            self.documentSelected.emit(None)
            return
        model: DocumentListModel = self.model()  # type: ignore[assignment]
        self.documentSelected.emit(model.document(idxs[0].row()))


Overwriting tik/ui/right/widgets/document_list.py


#### tik/ui/widgets/drop_zone.py

In [255]:
%%writefile tik/ui/widgets/drop_zone.py
from __future__ import annotations

import json
from typing import Optional, Any

from PyQt6.QtCore import pyqtSignal, Qt
from PyQt6.QtWidgets import QLabel, QWidget
from PyQt6.QtCore import QMimeData


class FieldDropZone(QLabel):
    """Drop target nhận payload JSON (text/plain hoặc application/json) từ QWebEngineView."""
    acceptRequested = pyqtSignal(object)  # emits DataChunk-like object (dict-like object)

    def __init__(self, field: str, parent: Optional[QWidget] = None):
        super().__init__(parent)
        self.field = field
        self.setText("Drop here")
        self.setAlignment(Qt.AlignmentFlag.AlignCenter)
        self.setAcceptDrops(True)
        self.setStyleSheet(
            "border: 1px dashed #2a3a4a; padding: 10px; border-radius: 8px; color: #93a4b8;"
        )

    # ---- helpers ----
    def _parse_mime(self, md: QMimeData) -> Optional[dict[str, Any]]:
        # 1) text/plain
        if md.hasText():
            try:
                return json.loads(md.text())
            except Exception:
                pass
        # 2) application/json (dự phòng)
        if md.hasFormat("application/json"):
            try:
                raw = md.data("application/json")  # QByteArray | bytes
                if hasattr(raw, "data"):
                    raw = raw.data()
                s = bytes(raw).decode("utf-8", errors="ignore")
                return json.loads(s)
            except Exception:
                pass
        return None

    # ---- DnD events ----
    def dragEnterEvent(self, e):
        data = self._parse_mime(e.mimeData())
        if data and data.get("field") == self.field:
            e.acceptProposedAction()
        else:
            e.ignore()

    def dragMoveEvent(self, e):
        data = self._parse_mime(e.mimeData())
        if data and data.get("field") == self.field:
            e.acceptProposedAction()
        else:
            e.ignore()

    def dropEvent(self, e):
        data = self._parse_mime(e.mimeData())
        if not data or data.get("field") != self.field:
            e.ignore()
            return

        payload = {
            "id": data["chunkId"],
            "document_id": data["documentId"],
            "source_id": data["sourceId"],
            "field": data["field"],
            "value": data["value"],
            "offset_start": data.get("offsetStart", 0),
            "offset_end": data.get("offsetEnd", 0),
            "exclusive_group": data.get("exclusiveGroup") or None,
        }
        # Phát object-like để Store.request_accept có thể truy cập thuộc tính (dot access)
        self.acceptRequested.emit(type("Obj", (), payload))
        e.acceptProposedAction()


Overwriting tik/ui/widgets/drop_zone.py


#### tik/ui/widgets/toast.py

In [256]:
%%writefile tik/ui/widgets/toast.py
from __future__ import annotations

from PyQt6.QtCore import QTimer, Qt, QPoint
from PyQt6.QtWidgets import QLabel, QWidget


class Toast(QLabel):
    def __init__(self, parent: QWidget):
        super().__init__(parent)
        self.setWindowFlags(Qt.WindowType.ToolTip)
        self.setStyleSheet(
            "background:#16202b; border:1px solid #2a3a4a; border-radius:8px; padding:8px 12px;"
        )
        self.hide()
        self._timer = QTimer(self)
        self._timer.setInterval(2200)
        self._timer.timeout.connect(self.hide)

    def show(self, text: str) -> None:  # type: ignore[override]
        super().setText(text)
        p = self.parent().mapToGlobal(QPoint(30, 30))
        super().move(p)
        super().show()
        self._timer.start()


Overwriting tik/ui/widgets/toast.py


#### tik/ui/overlays/objectives.py

In [257]:
%%writefile tik/ui/overlays/objectives.py
from __future__ import annotations

from PyQt6.QtCore import Qt
from PyQt6.QtWidgets import QDialog, QVBoxLayout, QLabel, QListWidget, QListWidgetItem, QPushButton

from ...core.store import Store
from ...core.models import Objective


class ObjectivesDialog(QDialog):
    def __init__(self, parent, store: Store):
        super().__init__(parent)
        self.setWindowTitle("Objectives")
        self.store = store
        lay = QVBoxLayout(self)
        lay.addWidget(QLabel("Complete these to progress the case:"))
        self.list = QListWidget()
        lay.addWidget(self.list)
        btn = QPushButton("Close")
        btn.clicked.connect(self.accept)
        lay.addWidget(btn)
        self._refresh()

    def _refresh(self) -> None:
        self.list.clear()
        case = self.store.sel.case
        if not case:
            return
        objectives = getattr(case, "_objectives", [])
        for obj in objectives:
            it = QListWidgetItem(f"{'✔' if obj.satisfied else '○'}  {obj.title}")
            it.setFlags(Qt.ItemFlag.ItemIsEnabled)
            self.list.addItem(it)


Overwriting tik/ui/overlays/objectives.py


#### tik/ui/overlays/conflicts.py

In [258]:
%%writefile tik/ui/overlays/conflicts.py
from __future__ import annotations

from PyQt6.QtWidgets import QDialog, QVBoxLayout, QLabel, QPushButton, QHBoxLayout

from ...core.models import AcceptedChunk, DataChunk


class ConflictsDialog(QDialog):
    def __init__(self, parent, current: AcceptedChunk, incoming: DataChunk):
        super().__init__(parent)
        self.setWindowTitle("Resolve Conflict")
        self._winner: AcceptedChunk | None = None

        lay = QVBoxLayout(self)
        lay.addWidget(QLabel(f"Field: {current.field}"))
        lay.addWidget(QLabel(f"Keep current: {current.value}"))
        lay.addWidget(QLabel(f"Use incoming: {incoming.value}"))

        btns = QHBoxLayout()
        keep = QPushButton("Keep current")
        use = QPushButton("Use incoming")
        cancel = QPushButton("Cancel")

        keep.clicked.connect(lambda: self._choose(current))
        use.clicked.connect(lambda: self._choose(AcceptedChunk(
            chunk_id=incoming.id, field=incoming.field, value=incoming.value,
            source_id=incoming.source_id, document_id=incoming.document_id,
            exclusive_group=incoming.exclusive_group)))
        cancel.clicked.connect(self.reject)

        btns.addWidget(keep); btns.addWidget(use); btns.addWidget(cancel)
        lay.addLayout(btns)

    def _choose(self, winner: AcceptedChunk) -> None:
        self._winner = winner
        self.accept()

    def winner(self) -> AcceptedChunk | None:
        return self._winner


Overwriting tik/ui/overlays/conflicts.py


#### tik/ui/overlays/logdock.py

In [259]:
%%writefile tik/ui/overlays/logdock.py
from __future__ import annotations

from PyQt6.QtWidgets import QDockWidget, QTextEdit
from PyQt6.QtCore import Qt

from ...core.models import AdvisorEvent


class LogDock(QDockWidget):
    def __init__(self, parent=None):
        super().__init__("Advisor Log", parent)
        self.setAllowedAreas(Qt.DockWidgetArea.BottomDockWidgetArea | Qt.DockWidgetArea.TopDockWidgetArea)
        self.view = QTextEdit(self)
        self.view.setReadOnly(True)
        self.setWidget(self.view)

    def append(self, evt: AdvisorEvent) -> None:
        self.view.append(f"[{evt.level.upper()}] {evt.text}")

    def append_text(self, text: str) -> None:
        self.view.append(text)


Overwriting tik/ui/overlays/logdock.py


#### tik/ui/web/base.html

In [260]:
%%writefile tik/ui/web/base.html
<!doctype html>
<html>
  <head>
    <meta charset="utf-8">
    <title>TIK Document</title>
    <link rel="stylesheet" href="style.css">
  </head>
  <body>
    <div id="container"></div>
    <script src="highlight.js"></script>
  </body>
</html>


Overwriting tik/ui/web/base.html


#### tik/ui/web/highlight.js

In [261]:
%%writefile tik/ui/web/highlight.js
(function () {
  function toPayload(span) {
    return JSON.stringify({
      chunkId: span.getAttribute("data-chunk-id"),
      field: span.getAttribute("data-field"),
      value: span.getAttribute("data-value"),
      sourceId: span.getAttribute("data-source-id"),
      documentId: span.getAttribute("data-document-id"),
      exclusiveGroup: span.getAttribute("data-exclusive-group") || null
    });
  }

  function setup() {
    document.querySelectorAll("span.chunk").forEach(function (sp) {
      // đảm bảo phần tử có thể kéo
      sp.setAttribute("draggable", "true");

      sp.addEventListener("dragstart", function (ev) {
        const payload = toPayload(sp);
        try { ev.dataTransfer.effectAllowed = "copyMove"; } catch (e) {}
        try { ev.dataTransfer.dropEffect = "copy"; } catch (e) {}
        // Qt thường đọc text/plain; thêm application/json làm dự phòng
        try { ev.dataTransfer.setData("text/plain", payload); } catch (e) {}
        try { ev.dataTransfer.setData("application/json", payload); } catch (e) {}
      });

      sp.addEventListener("mouseenter", function () { sp.classList.add("hover"); });
      sp.addEventListener("mouseleave", function () { sp.classList.remove("hover"); });
    });
  }

  if (document.readyState === "loading") {
    document.addEventListener("DOMContentLoaded", setup);
  } else {
    setup();
  }
})();


Overwriting tik/ui/web/highlight.js


#### tik/ui/web/style.css

In [262]:
%%writefile tik/ui/web/style.css
body {
  background: #0e141b;
  color: #d7e3f4;
  font-family: Segoe UI, Inter, Arial, sans-serif;
  line-height: 1.5;
  padding: 16px;
}
#container {
  background: #2f2f2f;
  border: 1px solid #3a3a3a;
  border-radius: 8px;
  padding: 16px;
}
span.chunk {
  background: rgba(88,166,255,0.12);
  border-bottom: 1px dashed #58a6ff;
  cursor: grab;
  -webkit-user-drag: element;  /* giúp Chromium coi đây là phần tử kéo được */
  user-select: none;           /* tránh bôi đen chữ khi kéo */
}
span.chunk.hover {
  background: rgba(88,166,255,0.22);
}


Overwriting tik/ui/web/style.css


#### tik/data/seed_case.json

In [263]:
%%writefile tik/data/seed_case.json
{
  "case": {
    "id": "case-0001",
    "title": "Case 0001 — Sample",
    "people": [
      { "id": "p-001", "name": null, "dob": null, "address": null, "occupation": null, "accepted": {} }
    ],
    "sources": [
      { "id": "s-001", "title": "Citizen DB" },
      { "id": "s-002", "title": "Social Feed" }
    ],
    "documents": [
      { "id": "doc_0001", "source_id": "s-001", "title": "Registry Entry", "path": "tik/data/docs/doc_0001.html" }
    ]
  },
  "objectives": [
    {
      "id": "obj-001",
      "title": "Identify subject's name and date of birth",
      "expr": {
        "kind": "AND",
        "children": [
          { "kind": "LEAF", "predicate": { "op": "exists", "path": "person.name" } },
          { "kind": "LEAF", "predicate": { "op": "exists", "path": "person.dob" } }
        ]
      },
      "satisfied": false
    }
  ]
}


Overwriting tik/data/seed_case.json


#### tik/data/docs/doc_0001.html

In [264]:
%%writefile tik/data/docs/doc_0001.html
<p>Citizen registry: Subject identified as <b>Jane Doe</b>, born on <i>1990-01-23</i>. 
Residence: <span>42 Evergreen Terrace</span>. Occupation listed as <span>Research Analyst</span>.</p>


Overwriting tik/data/docs/doc_0001.html


In [265]:
%%writefile tik/data/chunks/doc_0001.json
[
  {
    "id": "c-001",
    "document_id": "doc_0001",
    "source_id": "s-001",
    "field": "name",
    "value": "Jane Doe",
    "offset_start": 32,
    "offset_end": 40,
    "exclusive_group": "identity"
  },
  {
    "id": "c-002",
    "document_id": "doc_0001",
    "source_id": "s-001",
    "field": "dob",
    "value": "1990-01-23",
    "offset_start": 52,
    "offset_end": 62,
    "exclusive_group": "dob"
  },
  {
    "id": "c-003",
    "document_id": "doc_0001",
    "source_id": "s-001",
    "field": "address",
    "value": "42 Evergreen Terrace",
    "offset_start": 74,
    "offset_end": 95,
    "exclusive_group": "address"
  },
  {
    "id": "c-004",
    "document_id": "doc_0001",
    "source_id": "s-001",
    "field": "occupation",
    "value": "Research Analyst",
    "offset_start": 118,
    "offset_end": 133,
    "exclusive_group": "occupation"
  },
  {
    "id": "c-005",
    "document_id": "doc_0001",
    "source_id": "s-002",
    "field": "name",
    "value": "Jane A. Doe",
    "offset_start": 32,
    "offset_end": 40,
    "exclusive_group": "identity"
  }
]


Overwriting tik/data/chunks/doc_0001.json


## scripts/run_demo.py

In [266]:
%%writefile scripts/run_demo.py
from __future__ import annotations
import sys
from tik.app import main

if __name__ == "__main__":
    sys.exit(main())


Overwriting scripts/run_demo.py


## tests/test_models.py

In [267]:
%%writefile tests/test_models.py
from __future__ import annotations

from tik.core.models import DataChunk, AcceptedChunk, ObjectiveExpr, ObjectivePredicate, Objective


def test_chunk_roundtrip():
    c = DataChunk(
        id="c1", document_id="d1", source_id="s1",
        field="name", value="Alice", offset_start=10, offset_end=15, exclusive_group="identity"
    )
    ac = AcceptedChunk(
        chunk_id=c.id, field=c.field, value=c.value,
        source_id=c.source_id, document_id=c.document_id, exclusive_group=c.exclusive_group
    )
    assert ac.field == "name" and ac.value == "Alice"

def test_objective_tree():
    obj = Objective(
        id="o1", title="has name", satisfied=False,
        expr=ObjectiveExpr(kind="LEAF", predicate=ObjectivePredicate(op="exists", path="person.name"))
    )
    assert obj.expr.kind == "LEAF"


Overwriting tests/test_models.py


#### tests/test_store.py

In [268]:
%%writefile tests/test_store.py
from __future__ import annotations

from tik.core.store import Store
from tik.core.services.fake import FakeCaseService, FakeDocumentService, FakeChunkService, FakeObjectiveService, FakeEventService
from pathlib import Path


def _mk_store():
    data_dir = Path(__file__).resolve().parents[1] / "tik" / "data"
    return Store(
        case_service=FakeCaseService(data_dir),
        document_service=FakeDocumentService(data_dir),
        chunk_service=FakeChunkService(data_dir),
        objective_service=FakeObjectiveService(),
        event_service=FakeEventService()
    )


def test_accept_and_undo(qtbot):
    s = _mk_store()
    s.load_default_case()
    case = s.sel.case
    doc = case.documents[0]
    html, chunks = s.doc_svc.load_document_html_and_chunks(doc.id)  # type: ignore[attr-defined]
    # pick the name chunk
    c = [x for x in chunks if x["field"] == "name"][0]
    s.request_accept(type("Obj", (), {
        "id": c["id"], "document_id": c["document_id"], "source_id": c["source_id"],
        "field": c["field"], "value": c["value"], "offset_start": 0, "offset_end": 0,
        "exclusive_group": c.get("exclusive_group")
    }))
    assert "name" in s.sel.person.accepted
    s.undo_stack.undo()
    assert "name" not in s.sel.person.accepted


Overwriting tests/test_store.py


#### tests/test_fake_service.py

In [269]:
%%writefile tests/test_fake_service.py

from __future__ import annotations

from pathlib import Path
from tik.core.services.fake import FakeCaseService, FakeDocumentService, FakeChunkService, FakeObjectiveService
from tik.core.models import ObjectiveExpr, ObjectivePredicate, AcceptedChunk


def test_objective_eval_changes():
    data_dir = Path(__file__).resolve().parents[1] / "tik" / "data"
    case = FakeCaseService(data_dir).load_default_case()
    objsvc = FakeObjectiveService()
    person = case.people[0]
    # initially false
    changed = objsvc.evaluate(case, person)
    assert changed is False
    # set name (đúng kiểu AcceptedChunk)
    person.accepted["name"] = AcceptedChunk(
        chunk_id="x", field="name", value="Jane", source_id="s", document_id="d", exclusive_group="identity"
    )
    changed = objsvc.evaluate(case, person)
    assert isinstance(changed, bool)


Overwriting tests/test_fake_service.py


In [270]:
%%writefile scripts/how_to_run.py
print(r"""
=== TIK Phase 1 — Run Instructions ===

1) Create & activate a virtual environment:
   python -m venv .venv
   # Windows: .venv\Scripts\activate
   # macOS/Linux: source .venv/bin/activate

2) Install dependencies:
   pip install -U pip
   pip install -e .
   # or: pip install -r requirements.txt

3) Run the app:
   python -m tik.app
   # or:
   python scripts/run_demo.py

4) Run tests:
   pytest -q

Tips:
- If WebEngine fails to load on Linux, ensure Qt WebEngine runtime packages are installed.
- Use Undo/Redo from the toolbar after drag-and-drop to verify command stack.
""")


Overwriting scripts/how_to_run.py


In [271]:
%%writefile scripts/manual_test_checklist.py
print(r"""
=== TIK Phase 1 — Manual QA Checklist ===

[ ] Launch app (python -m tik.app). Window should show left tabs (Profiler/Graph) and Reader on right.
[ ] In Reader, select "Citizen DB" then "Registry Entry". The document renders highlighted chunks.
[ ] Drag the "Jane Doe" highlight onto Profiler's Name drop zone. Name should populate; a toast appears; Log shows entry.
[ ] Drag the date "1990-01-23" to DOB. Objective "Identify subject's name and date of birth" should now show as complete in Objectives dialog.
[ ] Drag the conflicting name (from Social Feed if present) onto Name. Conflict dialog appears; choose either option and verify Profiler updates accordingly.
[ ] Use Undo, then Redo from toolbar to confirm state transitions.
[ ] Retract a field via its button; verify it clears and can be undone.
[ ] Close and relaunch app (state is in-memory only; values reset), ensuring no crashes on load.

Notes:
- Drag payload is plain text JSON; verify with an external text drop target if debugging.
- If drops don't work, click inside the web view once; some platforms require focus before DnD.
""")


Overwriting scripts/manual_test_checklist.py
